# Download Sentinel-5P/TROPOMI observations
This example demonstrates how to download TROPOMI NO$_2$ fields around a list of sources using either the GUI or function calls.

In [ ]:
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import ddeq
import getpass

data_path_raw = os.path.join('data', 'S5P_raw')
data_path_cropped = os.path.join('data', 'S5P_cropped')

In [ ]:
# list of point sources
sources = ddeq.misc.read_point_sources()
source_list = ["Belchatow", "Janschwalde", "Lipetsk", "Matimba"]
sources

## Graphical user interface
*Acknowledgement*: The following UI is based on the [Python Script Tutorial](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/python_tropomi_level2_download.php) by the NOAA/NESDIS/STAR Aerosols and Atmospheric Composition Science Team.

The TROPOMI NO$_2$ product are available as "Near real time", "Offline" and "Reprocessing" product. The current version 02.04 is available as offline product (OFFL) from 25 July 2022. For earlier dates, the reprocessed data (RPRO) is available, which is recommended for orbits 24655 to 24779. However, at the time of writing (i.e. August 2023), the OFFL product is only available till orbit 22060 measured on 15 January 2022. For more information see: https://sentinel.esa.int/documents/247904/3541451/Sentinel-5P-Nitrogen-Dioxide-Level-2-Product-Readme-File

### Download example file
In the following, we demonstrate how to download and crop TROPOMI NO$_2$ around the Matimbe/Medupi power plant in South Africa. We only download files for 25 July 2021 that are also used for demonstration in other tutorials. 

The following code opens a GUI for downloading TROPOMI data. Select "Nitrogen dioxde" as product, "Reprocessing" as data latency and "L2" as processing level. For start and end date, we use 25 July 2021. Three options are availble for defining the area of interest. Here, we use "Point source" and choose "Matimba" from the list.

*Important: Do not re-run the cell below after picking your parameters, because this will reset values to default!*

In [ ]:
parameters = ddeq.download_S5P.show_UI(sources)

To download S5P/TROPOMI data, you need to create an account on the Copernicus Data Space Ecosystem (https://dataspace.copernicus.eu/).

In [ ]:
def get_user_credentials():
    username = input("Enter your username: ")
    password = getpass.getpass("Enter your password: ")
    return username, password

username, password = get_user_credentials()

In [ ]:
ddeq.download_S5P.main_UI(username, password, data_path_raw, *parameters, sources)

### Command line version
The API can also be used without the script using the function `get_tropomi_files`. 

First, the polygon used for selcting the region around a source can be calculated with the function `get_bounding_box`. It uses the coordinates of a point and a distance in meters to calculate a polygon using the coordinates as center and distance as radius.

In [ ]:
lon_o, lat_o = ddeq.misc.get_source_location(sources.sel(source='Matimba'))
west_lon, east_lon, south_lat, north_lat = ddeq.download_S5P.get_bounding_box(lon_o, lat_o, 1e3)

The query parameters can be inserted manually in the following format:

In [ ]:
ids, filenames = ddeq.download_S5P.list_files(
        west_lon, east_lon, south_lat, north_lat,
        start_date='2021-07-25',
        end_date='2021-07-25',
        product_abbreviation='L2__NO2___',
        latency='RPRO',
        level='L2',
        orbit='-'
)

# create download and start downloading
downloader = ddeq.download_S5P.Download(data_path_raw, username, password)
downloader.download_files(ids, filenames)

Next, one has to create a list of all S5P files which should be processed further.

In [ ]:
all_files = sorted(glob.glob(os.path.join(data_path_raw, '*NO2*.nc')))
all_files = [os.path.basename(file) for file in all_files]
all_files

### Plot orbits

To quickly check the coverage of an image, one can use the `plot_orbit` function.

In [ ]:
ddeq.download_S5P.plot_orbit(all_files[0], path=data_path_raw)

### Crop and save
To crop the swath around the source of interest, `ddeq.download_S5P.crop_and_save` can be used as follows:

In [ ]:
ddeq.download_S5P.crop_and_save(all_files, sources.sel(source=["Matimba"]), distance=300e3, delete=False,
                                path_open=data_path_raw, path_save=data_path_cropped)

### Plot cropped image
The image can be visualized as follows:

In [ ]:
for filename in all_files:
    fn = os.path.join(data_path_cropped, f'Matimba_{filename}')
    data_S5p = xr.open_dataset(fn)

    fig = ddeq.download_S5P.plot_extent(data_S5p, 'NO2', sources, 0, 200e-6)